# Sign Language Recognition - Training Notebook

This notebook automatically trains the GRU model for sign language recognition.

## Quick Start:
1. **Runtime → Change runtime type → Select GPU**
2. **Runtime → Run all**

Or click the "Open in Colab" badge in the GitHub repository!

---

**Note:** Make sure your data is available in the `Data/` directory. If you need to upload data, see `docs/COLAB_UPLOAD_GUIDE.md`.


In [ ]:
# Clone repository
!git clone https://github.com/MAya0M/SignLanguage-Recognition.git
%cd SignLanguage-Recognition


In [ ]:
# Check GPU availability
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))


## Install Dependencies


In [ ]:
# Install required packages
%pip install -q tensorflow numpy pandas scikit-learn opencv-python mediapipe tqdm


## Verify Data

Make sure your data is in the `Data/` directory. If not, upload it using one of the methods in `docs/COLAB_UPLOAD_GUIDE.md`.


In [ ]:
# Verify data exists and check dataset size
import os
import pandas as pd
from pathlib import Path

data_dir = Path('Data')
if data_dir.exists():
    print("✅ Data directory found")
    csv_path = data_dir / 'Labels' / 'dataset.csv'
    keypoints_dir = data_dir / 'Keypoints' / 'rawVideos'
    
    if csv_path.exists():
        print(f"✅ CSV file: {csv_path}")
        # Check dataset size
        df = pd.read_csv(csv_path)
        print(f"\n📊 Dataset Statistics:")
        print(f"   Total samples: {len(df)}")
        print(f"   Expected: 226 samples (with new videos)")
        
        if len(df) < 200:
            print(f"\n⚠️  WARNING: Only {len(df)} samples found!")
            print(f"   Expected 226 samples. Make sure CSV is updated with new videos.")
            print(f"   Run: !python scripts/create_dataset_csv.py")
        else:
            print(f"   ✅ CSV has correct number of samples!")
        
        print(f"\n   Per label:")
        for label, count in df.groupby('label').size().sort_values(ascending=False).items():
            status = "✅" if count >= 25 else "⚠️"
            print(f"      {status} {label:12s}: {count:3d} samples")
        
        print(f"\n   Per split:")
        for split, count in df.groupby('split').size().items():
            print(f"      {split:6s}: {count:3d} samples")
    else:
        print(f"❌ CSV file not found: {csv_path}")
    
    if keypoints_dir.exists():
        print(f"\n✅ Keypoints directory: {keypoints_dir}")
        # Count keypoint files
        npy_files = list(keypoints_dir.rglob("*.npy"))
        print(f"   Found {len(npy_files)} keypoint files")
        if len(npy_files) < 200:
            print(f"   ⚠️  Expected ~226 keypoint files")
    else:
        print(f"❌ Keypoints directory not found: {keypoints_dir}")
else:
    print("❌ Data directory not found")
    print("Please upload data first! See docs/COLAB_UPLOAD_GUIDE.md")

# Run detailed dataset check
print("\n" + "="*60)
print("Running detailed dataset verification...")
print("="*60)
!python scripts/check_dataset.py


In [ ]:
## Debug: Check Why Model Is Not Learning

If model accuracy stays at 12.5% (random chance), run this to diagnose:


In [ ]:
# Run debug script to find why model is not learning
# This will check:
# - If data loads correctly
# - If labels are balanced
# - If data has variation
# - If model can learn at all

!python scripts/debug_model_training.py


## Train Model


In [ ]:
# Train the model with optimized parameters for better learning
# Improved settings:
# - Lower learning rate (0.0005) - slightly higher for faster learning
# - Smaller batch size (16) for better gradient updates
# - More GRU units (256) and layers (3) for better capacity
# - Moderate dropout (0.3) - reduced to allow more learning
# - More patience (50) - will be tripled internally (150 epochs) to allow model to learn much longer
# - More epochs (200) to give model more time to converge
# - Early stopping now monitors val_accuracy with very small min_delta to catch any improvement
# - Removed batch normalization from dense layers (can interfere with small datasets)

!python scripts/train_model.py --csv Data/Labels/dataset.csv --keypoints-dir Data/Keypoints/rawVideos --output-dir models --batch-size 16 --epochs 200 --gru-units 256 --num-gru-layers 3 --dropout 0.3 --learning-rate 0.0005 --patience 50


## Evaluate Model Quality

After training, check the prediction quality:


In [ ]:
# Evaluate the latest trained model
import glob
from pathlib import Path

models = sorted(glob.glob('models/run_*/best_model.keras'))
if models:
    latest_model = models[-1]
    print(f"📊 Evaluating: {Path(latest_model).name}")
    print("=" * 60)
    
    # Run evaluation
    !python scripts/evaluate_model.py --model {latest_model} --csv Data/Labels/dataset.csv --keypoints-dir Data/Keypoints/rawVideos
else:
    print("❌ No models found - train the model first!")


## Download Model (Optional)

To save your trained model to Google Drive:


In [ ]:
# Download model to your computer
from google.colab import files
import shutil
import glob
from pathlib import Path

models_dir = sorted(glob.glob('models/run_*'))
if models_dir:
    latest_run = models_dir[-1]  # Latest run
    print(f"📦 Preparing model: {Path(latest_run).name}")
    
    # Create a zip file
    zip_name = f"{Path(latest_run).name}"
    shutil.make_archive(zip_name, 'zip', latest_run)
    
    # Download
    print(f"⬇️ Downloading {zip_name}.zip...")
    files.download(f'{zip_name}.zip')
    print("✅ Model downloaded! Extract and add to your GitHub repo.")
else:
    print("❌ No models found - train the model first!")

# Alternative: Save to Google Drive (uncomment to use)
# from google.colab import drive
# drive.mount('/content/drive')
# dest = f'/content/drive/MyDrive/{Path(latest_run).name}'
# shutil.copytree(latest_run, dest, dirs_exist_ok=True)
# print(f"✅ Model saved to Google Drive: {Path(latest_run).name}")
